# [Module 1.1] 데이터 준비

이 노트북은 주로 아래와 같은 작업을 합니다.
- 훈련 데이터 세트 로딩
- 데이터 샘플링하여 훈련, 테스트 데이터 세트 생성
- 데이터 변형
- 데이터 저장
- 변수 저장
---


# 1. 환경 셋업
- 아래는 파이썬 캐키지를 임포트할때에 캐싱된 것을 사용하지 않고, 매번 리로딩 하는 세팅 입니다.

In [1]:
%load_ext autoreload
%autoreload 2

- 데이터 위치 지정

In [2]:
import pandas as pd
import os

data_folder = '../../../data/AdTalking'

# 2. 훈련 데이터 로딩 및 데이터 정리
- 아래는 데이터 로딩을 위해서 9.6 GB 가 필요합니다. 메모리 할당 에러가 나면 다른 노트북의 커널 세션을 삭제하고 해보세요. 그래도 안되면 메모리가 많은 노트북 인스턴스에서 다시 하시기 바랍니다.

In [3]:
%%time
file = 'train.csv'
file_path = os.path.join(data_folder, file)

df = pd.read_csv(file_path, parse_dates=['click_time'])
df

CPU times: user 1min 28s, sys: 10.4 s, total: 1min 38s
Wall time: 1min 38s


,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,83230,3,1,13,379,2017-11-06 14:32:21,NaN,0
1,17357,3,1,19,379,2017-11-06 14:33:34,NaN,0
2,35810,3,1,13,379,2017-11-06 14:34:12,NaN,0
3,45745,14,1,13,478,2017-11-06 14:34:52,NaN,0
4,161007,3,1,13,379,2017-11-06 14:35:08,NaN,0
...,...,...,...,...,...,...,...,...
184903885,121312,12,1,10,340,2017-11-09 16:00:00,NaN,0
184903886,46894,3,1,19,211,2017-11-09 16:00:00,NaN,0
184903887,320126,1,1,13,274,2017-11-09 16:00:00,NaN,0
184903888,189286,12,1,37,259,2017-11-09 16:00:00,NaN,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184903890 entries, 0 to 184903889
Data columns (total 8 columns):
 #   Column           Dtype         
---  ------           -----         
 0   ip               int64         
 1   app              int64         
 2   device           int64         
 3   os               int64         
 4   channel          int64         
 5   click_time       datetime64[ns]
 6   attributed_time  object        
 7   is_attributed    int64         
dtypes: datetime64[ns](1), int64(6), object(1)
memory usage: 11.0+ GB


## 불필요 컬럼 드랍
- attributed_time 는 널 데이터가 많아서 사용하지 않음

In [5]:
def drop_column(raw_df, col):
    df = raw_df.drop(columns=[col])
    return df

df = drop_column(df, col='attributed_time')

## 컬럼 이름 변경
- 아래의 레이블 및 click_time은 AFD 에서 지정한 컬럼 이름으로 변경 합니다.

In [6]:
df = df.rename(columns={'is_attributed':'EVENT_LABEL'})    
df = df.rename(columns={'click_time':'EVENT_TIMESTAMP'})    


# 3. 데이타 Resampling (훈련, 테스트)

- 주어진 데이터의 시작 날짜와 끝 날짜를 확인 합니다.

In [7]:
print("min time: ", df.EVENT_TIMESTAMP.min())
print("max time: ", df.EVENT_TIMESTAMP.max())

min time:  2017-11-06 14:32:21
max time:  2017-11-09 16:00:00


## 훈련 및 테스트 샘플링 함수
- 데이터 세트를 train_end 보다 작은 것을 훈련 데이터 세트, test_start 큰 것을 테스트 데이터 세트로 분리 합니다.
- 전체 샘플링 개수를 기술 합니다. (에: total_samples = 200,000)
- 샘플링 데이터 세트에서 훈련, 테스트 분리의 비율을 기술 합니다. 
    - (예: split_rate: 0.1)
        - 훈련 90%, 테스트 10%
- 결과로서 훈련, 테스트 샘플 데이터 세트에서 프로드의 비율 및 개수를 확인 합니다.        
    

In [8]:
def split_data_by_time(df, target_col, label_col, total_samples, split_rate, train_end, test_start, verbose=False):
    '''
    시간 관점으로 번반부튼 훈련, 후반부는 테스트 데이터로 해서 샘블링 함.
    '''
    
    # 훈련 데이터 셋
    train_df = df[df[target_col] <= train_end]   
    train_num = int(total_samples * (1 - split_rate))    # 훈련 샘플 데이터 수
    train_sample = train_df.sample(n = train_num, random_state=100)    # 샘플링    

    print("train sample shape: ", train_sample.shape)
    print("train min time: ", train_sample[target_col].min())
    print("train max time: ", train_sample[target_col].max())
    print("Train fraud ratio: ", round(train_sample[label_col].value_counts()[1] / train_sample.shape[0],5))
    print("# of Train frauds: ", train_sample[label_col].value_counts()[1])     


    # 테스트 데이터 셋    
    test_df = df[df[target_col] >= test_start]    
    test_num = int(total_samples * (split_rate))    # 테스트 샘플 데이터 수
    test_sample = test_df.sample(n = test_num, random_state=100)    
    

    print("\ntest sample shape: ", test_sample.shape)    
    print("test min time: ", test_sample[target_col].min())
    print("test max time: ", test_sample[target_col].max())
    print("Test fraud ratio: ", round(test_sample[label_col].value_counts()[1] / test_sample.shape[0],5))    
    print("# of test frauds: ", test_sample[label_col].value_counts()[1])         
    
    
    return train_sample, test_sample
    
train_df, test_df = split_data_by_time(
                       df=df, 
                       target_col='EVENT_TIMESTAMP', 
                       label_col = 'EVENT_LABEL',
                       total_samples=200000, 
                       split_rate=0.1, 
                       train_end='2017-11-08 23:59', 
                       test_start='2017-11-09 00:00',    
                       verbose = True,
                  )    

train sample shape:  (180000, 7)
train min time:  2017-11-06 15:08:24
train max time:  2017-11-08 23:58:58
Train fraud ratio:  0.00242
# of Train frauds:  435

test sample shape:  (20000, 7)
test min time:  2017-11-09 00:00:01
test max time:  2017-11-09 15:59:58
Test fraud ratio:  0.00275
# of test frauds:  55


# 4. 데이터 변형

## 코드 데이터 (예: OS) 문자형 으로 변경
- 숫자값에 'str' 를 넣어서 명시적으로 스트링으로 타입 변환
이유는 csv로 데이터 프레임을 저장시에, string 타입이 일반 숫자형 타입으로 변경이 되어서 명시적으로 스트링으로 변경 함.

In [9]:
from src.p_utils import change_code_to_string

train_df = change_code_to_string(train_df, col='ip', new_col='str_ip', verbose=False)
train_df = change_code_to_string(train_df, col='app', new_col='str_app', verbose=False)
train_df = change_code_to_string(train_df, col='device', new_col='str_device', verbose=False)
train_df = change_code_to_string(train_df, col='os', new_col='str_os', verbose=False)
train_df = change_code_to_string(train_df, col='channel', new_col='str_channel', verbose=False)

test_df = change_code_to_string(test_df, col='ip', new_col='str_ip', verbose=False)
test_df = change_code_to_string(test_df, col='app', new_col='str_app', verbose=False)
test_df = change_code_to_string(test_df, col='device', new_col='str_device', verbose=False)
test_df = change_code_to_string(test_df, col='os', new_col='str_os', verbose=False)
test_df = change_code_to_string(test_df, col='channel', new_col='str_channel', verbose=False)

### 기존 컬럼 삭제
- 위에서 같은 내용의 컬럼을 추가해서, 기존 컬럼은 삭제 함.

In [10]:
train_df = drop_column(train_df, col='ip')
train_df = drop_column(train_df, col='app')
train_df = drop_column(train_df, col='device')
train_df = drop_column(train_df, col='os')
train_df = drop_column(train_df, col='channel')

test_df = drop_column(test_df, col='ip')
test_df = drop_column(test_df, col='app')
test_df = drop_column(test_df, col='device')
test_df = drop_column(test_df, col='os')
test_df = drop_column(test_df, col='channel')

In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180000 entries, 37517593 to 62920975
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   str_ip           180000 non-null  object        
 1   str_app          180000 non-null  object        
 2   str_device       180000 non-null  object        
 3   str_os           180000 non-null  object        
 4   str_channel      180000 non-null  object        
 5   EVENT_TIMESTAMP  180000 non-null  datetime64[ns]
 6   EVENT_LABEL      180000 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 11.0+ MB


# 5. 데이터 저장

## 로컬에 데이터 CSV 로 저장
- 로컬에 훈련 및 테스트 데이터 세트 저장
    - data/train/train.csv
    - data/test/test.csv

In [12]:
from src.p_utils import save_csv_local

train_file_name = 'train-' + str(train_df.shape[0]) + ".csv"
train_local_path = save_csv_local(raw_df=train_df, preproc_folder='data/train', 
                                  label='EVENT_LABEL', file_name=train_file_name)
print("train_local_path: ", train_local_path)

test_file_name = 'test-' + str(test_df.shape[0]) + ".csv"
test_local_path = save_csv_local(raw_df=test_df, preproc_folder='data/test', 
                                  label='EVENT_LABEL', file_name=test_file_name)
print("test_local_path: ", test_local_path)

data/train/train-180000.csv is saved
train_local_path:  data/train/train-180000.csv
data/test/test-20000.csv is saved
test_local_path:  data/test/test-20000.csv


## S3에 데이로 업로딩
- 위의 파일을 S3에 업로드

In [13]:
import sagemaker

bucket = sagemaker.Session().default_bucket()
# 프로젝트 변수
project_prefix = 'adtalking_fraud_phase0'


# S3에 저장되는 데이터의 기본 폴더 위치
s3_train_data_uri = f"s3://{bucket}/{project_prefix}/train"
s3_test_data_uri = f"s3://{bucket}/{project_prefix}/test"


In [14]:
s3_train_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=train_local_path, 
    desired_s3_uri=s3_train_data_uri,    
)
print("s3_train_data_uri: \n", s3_train_data_uri)

s3_test_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=test_local_path, 
    desired_s3_uri=s3_test_data_uri,    
)
print("s3_test_data_uri: \n", s3_test_data_uri)



s3_train_data_uri: 
 s3://sagemaker-us-east-1-057716757052/adtalking_fraud_phase0/train/train-180000.csv
s3_test_data_uri: 
 s3://sagemaker-us-east-1-057716757052/adtalking_fraud_phase0/test/test-20000.csv


# 6. 변수 저장
- 다른 노트북에서 아래 변수의 값을 사용하기 위해서 저장 합니다.

In [15]:
%store project_prefix
%store bucket
%store train_local_path
%store test_local_path
%store s3_train_data_uri
%store s3_test_data_uri

Stored 'project_prefix' (str)
Stored 'bucket' (str)
Stored 'train_local_path' (str)
Stored 'test_local_path' (str)
Stored 's3_train_data_uri' (str)
Stored 's3_test_data_uri' (str)
